# Data cleaning and merging dataframes

## Loading multiple datasets

### Classical way

Reading file by file

In [168]:
import pandas as pd

path = 'data/eniac/'
orderlines = pd.read_csv(path + 'orderlines.csv')
orders = pd.read_csv(path + 'orders.csv')
brands = pd.read_csv(path + 'brands.csv')
products = pd.read_csv(path + 'products.csv')

df_list = [orderlines, orders, brands, products]
files = ['orderlines','orders','brands','products']

### Another way

Using `os` and a loop to read all the files from a directory. It's also possible to read only files with a certain extension (like `.csv`):

In [169]:
# import pandas as pd
# import os
# path = '../data/eniac/'
# path, dirs, files = next(os.walk(path))
# #print(files)

# # remove non-csv files
# for file in files:
#     if not file.endswith("csv"):
#         files.remove(file)

In [170]:
# # create empty list 
# df_list = []

# # append datasets to the list
# for file in files:
#     temp_df = pd.read_csv(path + file, sep=',')
#     df_list.append(temp_df)

# # show results
# products, orders, orderlines, brands = df_list[0], df_list[1], df_list[2], df_list[3]

## Data quality

### Missing values

In [171]:
def check_missing_values(data): 
    print('Missing values:' + '\n' + str(data.isna().sum()))

In [172]:
for i in range(len(df_list)): 
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    print('\n')

File name: orderlines
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64


File name: orders
Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64


File name: brands
Missing values:
short    0
long     0
dtype: int64


File name: products
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64




### Duplicates

In [173]:
def check_duplicates(data): 
    print('Duplicated rows: ', data.duplicated().sum())

In [174]:
for i in range(len(df_list)): 
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    check_duplicates(df_list[i])
    print('\n')

File name: orderlines
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64
Duplicated rows:  0


File name: orders
Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64
Duplicated rows:  0


File name: brands
Missing values:
short    0
long     0
dtype: int64
Duplicated rows:  0


File name: products
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64
Duplicated rows:  8746




Based on our initial exploration we need to find a way to deal with our missing values. 

Do we have other problems like data inconsistency? 

* Example: problems with the `price` column in the `orderlines` dataframe.

### Data inconsistency

In [175]:
for i in range(len(df_list)): 
    
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    check_duplicates(df_list[i])
    df_list[i].info()
    print('\n')

File name: orderlines
Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64
Duplicated rows:  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


File name: orders
Missing values:
order_id        0
created_date    0
total_paid      5
state           0
dtype: int64
Duplicated rows:  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 

Some aspects that do not make a lot of sense:

* **products**: 
    * `price` and `promo_price` are loaded as objects. They should be floats. 
    * Quite a lot of missing values: 
        * `description`: 7 missing values. Maybe that could be inferred from the product name?
        * `price`: the missing values could be filled from the `orderlines` dataset. But first we will need to clean it. 

    
* **orders**: 
    * `created_date` should have a date data type. Then, it would be a good quality check to see if the created date for `orders` mathces with the created dates for `orderlines`. 
    
    
* **orderlines**: 
    * `unit_price` has to be a float, something wrong there. 
    * `date` has to be transformed to a date data type. Then, as we said, check that it with matches with the `orders` dataset. 
    
    
* **brands**: looks fine. 


Where do we have to start? Since `orders` and `orderlines` seem very crucial to the analysis, we will start by checking that they match.

## Cleaning orders

In [176]:
# change date datatype
orders['created_date'] = pd.to_datetime(orders['created_date'])

Some data I could extract from here to do a data quality check with `orderlines`: 

* the sum of `total_paid` on both datasets should be the same
* the initial and last date should be the same

In [177]:
min(orders['created_date'])

Timestamp('2017-01-01 00:07:19')

In [178]:
max(orders['created_date'])

Timestamp('2018-03-14 13:58:36')

In [179]:
sum(orders['total_paid'])

nan

I forgot to deal with missing values on our orders dataset!

In [180]:
# let's filter them out
orders.loc[orders['total_paid'].isna()]

,order_id,created_date,total_paid,state
127701,427314,2017-11-20 18:54:39,NaN,Pending
132013,431655,2017-11-22 12:15:24,NaN,Pending
147316,447411,2017-11-27 10:32:37,NaN,Pending
148833,448966,2017-11-27 18:54:15,NaN,Pending
149434,449596,2017-11-27 21:52:08,NaN,Pending


These orders have a state of "Pending", so I can exclude them from our dataset. We are interested on the completed ones.

In [181]:
orders.dropna(inplace=True)

Now my dataset is clean. And the total paid is: 

In [182]:
orders['total_paid'].sum()


129159615.07000001

In [183]:
orders.shape

(226904, 4)

## Cleaning orderlines

First let's transform our date time. 

In [184]:
orderlines['date'] = pd.to_datetime(orderlines['date'])

In [185]:
min(orderlines['date'])
# orders: Timestamp('2017-01-01 00:07:19')

Timestamp('2017-01-01 00:07:19')

In [186]:
max(orderlines['date'])
# orders: Timestamp('2018-03-14 13:58:36')

Timestamp('2018-03-14 13:58:36')

Yes, it seems it match. Now let's test our total_paid. 

To get this value, we will have to calculate a new column, total price for each row. It would be `product_quantity` * `unit_price`. I can not multiply a two different types of data. So I will have to transform the unit price to a numeric one. 

In [187]:
# orderlines['product_quantity'] * pd.to_numeric(orderlines['unit_price'])

While trying to transform this column to numeric an error appears. From the error message, "Unable to parse string "1.137.99" at position 6", it seems that our dataset has some problems with the thousands separators: they were encoded as dots, and python only admits one dot per number: the _decimal_ separator!

There are a lot of different ways to solve this problem. The first thing I will do is to count how many dots appear for each number using string operations. If price has more than one dot, I will consider it corrupted. 

In [188]:
# we create a copy of the dataset
ol_temp = orderlines.copy()

# create a new column with the amount of dots in the unit_price column
ol_temp['dots'] = orderlines['unit_price'].str.count('\.')

# show the rows with more than one dot
ol_temp.query('dots > 1')

,id,id_order,product_id,product_quantity,sku,unit_price,date,dots
6,1119115,299544,0,1,APP1582,1.137.99,2017-01-01 01:17:21,2
11,1119126,299549,0,1,PAC0929,2.565.99,2017-01-01 02:07:42,2
15,1119131,299553,0,1,APP1854,3.278.99,2017-01-01 02:14:47,2
43,1119195,299582,0,1,PAC0961,2.616.99,2017-01-01 08:54:00,2
59,1119214,299596,0,1,PAC1599,2.873.99,2017-01-01 09:53:11,2
...,...,...,...,...,...,...,...,...
293862,1649999,452946,0,1,APP2075,2.999.00,2018-03-14 13:03:33,2
293887,1650045,527321,0,1,PAC2148,3.497.00,2018-03-14 13:10:15,2
293889,1650050,527324,0,1,PAC2117,3.075.00,2018-03-14 13:10:56,2
293911,1650088,527342,0,1,APP2492,1.329.00,2018-03-14 13:24:51,2


Our theory about the thousands separators is confirmed. How can we solve this problem? 

My approach would be to remove all the dots for all the unit_prices, and then add a dot before the last 2 digits to all the rows. Then I will transform it into numeric values.

In [189]:
orderlines = orderlines.assign(unit_price_nd = orderlines['unit_price'].str.replace('\.','', regex=True))
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374


In [190]:
orderlines['digits'] = orderlines['unit_price_nd'].str[-2:]
orderlines['integers'] = orderlines['unit_price_nd'].str[:-2]
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,digits,integers
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899,99,18
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900,00,399
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405,05,474
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839,39,68
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374,74,23


In [191]:
orderlines['new_unit_price'] = orderlines['integers'] + '.' + orderlines['digits']
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,digits,integers,new_unit_price
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899,99,18,18.99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900,00,399,399.00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405,05,474,474.05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839,39,68,68.39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374,74,23,23.74


In [192]:
# last step is to transform it to numeric
orderlines['unit_price'] = pd.to_numeric(orderlines['new_unit_price'])
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
 7   unit_price_nd     293983 non-null  object        
 8   digits            293983 non-null  object        
 9   integers          293983 non-null  object        
 10  new_unit_price    293983 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(5)
memory usage: 24.7+ MB


Now it is time to see if it is the same than the orders files. 

In [193]:
orderlines.drop(['unit_price_nd','digits','integers','new_unit_price'], axis=1, inplace=True)

orderlines['total_price'] = orderlines['product_quantity'] * orderlines['unit_price']
sum(orderlines['total_price'])
# 129159615.07000001


128776222.0298095

In [194]:
orderlines

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,18.99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,399.00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,474.05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,68.39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,23.74
...,...,...,...,...,...,...,...,...
293978,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25,42.99
293979,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34,141.58
293980,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,19.98
293981,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01,19.99


What!? After all this work the values do not match! How can we figure it out where is the problem? 

## Matching `orders` and `orderlines`

It is possible that some orders exist in one dataset but not in the other one. This would be a potential source for this price mismatch. Let's solve it!

We first create a new column in the `orderlines` dataset using `assign`. We also use `isin()` to create a boolen value (True/False) that checks whether the `id_order` is present in the `orders` dataset:

In [195]:
orderlines.assign(check_orders = orderlines['id_order'].isin(orders['order_id']))

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,18.99,True
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,399.00,True
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,474.05,True
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,68.39,True
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,23.74,True
...,...,...,...,...,...,...,...,...,...
293978,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25,42.99,True
293979,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34,141.58,True
293980,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,19.98,True
293981,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01,19.99,True


Then, using `.query` we select rows where the value in this new column is `False`:

In [196]:
(
orderlines
    .assign(check_orders = orderlines['id_order'].isin(orders['order_id']))
    .query("check_orders==False")
)

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders
5,1119114,295310,0,10,WDT0249,231.79,2017-01-01 01:14:27,2317.90,False
63,1119218,296284,0,1,BNQ0042,699.00,2017-01-01 09:58:35,699.00,False
67,1119223,294806,0,1,APP1849,2558.99,2017-01-01 10:09:15,2558.99,False
69,1119226,294806,0,1,APP1864,2797.99,2017-01-01 10:15:14,2797.99,False
70,1119235,297261,0,1,QNA0177,304.99,2017-01-01 10:17:59,304.99,False
...,...,...,...,...,...,...,...,...,...
275665,1621177,244328,0,1,OWC0260,349.00,2018-02-20 13:00:45,349.00,False
280055,1628699,261391,0,1,APP2352,3343.00,2018-02-27 11:43:25,3343.00,False
280352,1629247,287797,0,1,SYN0182,484.11,2018-02-27 19:16:26,484.11,False
280856,1630150,261391,0,1,SAT0091,44.99,2018-02-28 12:18:26,44.99,False


It looks like 234 rows in `orderlines` come from orders not present in the `orders` dataset. This is quite inconsistent, since the `orders` dataset should be the one and only source of truth for orders: if an order is not there, it should not exist. We will try to find out why that happened, but for now, let's just remove those "ghost" orders:

In [197]:
orderlines = (orderlines
              .assign(check_orders = orderlines['id_order'].isin(orders['order_id']))
              .query("check_orders==True"))

In [198]:
orderlines

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,18.99,True
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,399.00,True
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,474.05,True
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,68.39,True
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,23.74,True
...,...,...,...,...,...,...,...,...,...
293978,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25,42.99,True
293979,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34,141.58,True
293980,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,19.98,True
293981,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01,19.99,True


Now let's look at this problem in the opposite direction: are there orders in the `orders` dataset not prsent in `orderlines`?

In [199]:
(orders
 .assign(check_orders = orders['order_id'].isin(orderlines['id_order']))
 .query("check_orders==False"))

,order_id,created_date,total_paid,state,check_orders
8,245941,2017-01-01 10:32:23,183.52,Completed,False
65,268629,2017-01-31 11:27:25,73.23,Completed,False
75,272862,2017-04-18 18:17:10,128.99,Pending,False
82,277655,2017-02-01 08:44:55,9.99,Pending,False
83,277994,2017-01-23 18:30:11,52.99,Completed,False
...,...,...,...,...,...
226835,527328,2018-03-14 13:14:12,0.00,Shopping Basket,False
226851,527344,2018-03-14 13:25:21,0.00,Place Order,False
226853,527346,2018-03-14 13:26:14,0.00,Place Order,False
226855,527348,2018-03-14 13:28:18,0.00,Place Order,False


There are more than 22000 orders in the `orders` dataset that are not present on the `orderlines` dataset!!! We can try to find out why by looking at the state of these orders:

In [200]:
(orders
 .assign(check_orders = orders['order_id'].isin(orderlines['id_order']))
 .query("check_orders==False")
 ['state'].value_counts())

Place Order        12304
Shopping Basket     9810
Completed             45
Cancelled             41
Pending               13
Name: state, dtype: int64

It looks like most of them are orders that were not fully completed: the products were left in the shopping basket or the order was "placed" but maybe not paid (hence the state "Place Order". Some of them were "Completed", though. 

This will require further research, and we might have to come back to these orders if we have to explore consumer behaviour (e.g. why are orders left in the shopping basket?), but for now, for the sake of data quality, let's drop all of these unmatched orders:

In [201]:
orders = (orders
          .assign(check_orders = orders['order_id'].isin(orderlines['id_order']))
          .query("check_orders==True")
         )

Let's now check again if the total paid matches:

In [202]:
orders['total_paid'].sum()

129130877.18

In [203]:
orderlines['total_price'].sum()

128659393.77

STILL NOT MATCHING!!! This is outrageous. Let's keep exploring.

## Solving the price mismatch

Let's merge both datasets and compare, order by order, the `total_price`. We will call this new merged dataset `orders_info`.

*Note: To merge `orderlines` with `orders`, we have to group and aggregate `orderlines`!

In [204]:
orders_info = (
orderlines
    .groupby('id_order')
    .agg({'total_price':'sum'})
    .merge(orders, how='left', left_on='id_order', right_on='order_id')
    .copy()
)
orders_info

,total_price,order_id,created_date,total_paid,state,check_orders
0,44.99,241319,2017-01-02 13:35:40,44.99,Cancelled,True
1,129.16,241423,2017-11-06 13:10:02,136.15,Completed,True
2,10.77,242832,2017-12-31 17:40:03,15.76,Completed,True
3,77.99,243330,2017-02-16 10:59:38,84.98,Completed,True
4,153.87,243784,2017-11-24 13:35:19,157.86,Cancelled,True
...,...,...,...,...,...,...
204686,42.99,527397,2018-03-14 13:56:38,42.99,Place Order,True
204687,42.99,527398,2018-03-14 13:57:25,42.99,Shopping Basket,True
204688,141.58,527399,2018-03-14 13:57:34,141.58,Shopping Basket,True
204689,19.98,527400,2018-03-14 13:57:41,19.98,Shopping Basket,True


Now that the `total` from both datasets is in the same dataframe, we can create a new column with the difference:

In [205]:
orders_info['price_difference'] = orders_info['total_price'] - orders_info['total_paid']
orders_info.sort_values('price_difference').tail(30)

,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
20654,164.90,322147,2017-02-10 11:56:59,125.99,Completed,True,38.91
11725,164.90,312311,2017-01-20 00:59:54,125.99,Completed,True,38.91
45940,44.99,349377,2017-04-24 21:42:28,4.99,Pending,True,40.00
8545,164.90,308825,2017-01-13 14:27:46,123.99,Completed,True,40.91
13784,164.90,314570,2017-01-25 09:01:35,122.99,Completed,True,41.91
21347,164.90,322901,2017-02-12 13:25:12,122.99,Place Order,True,41.91
25517,164.90,327363,2017-02-21 22:42:17,122.99,Completed,True,41.91
986,164.90,300474,2017-01-02 12:24:54,119.00,Completed,True,45.90
166458,214768.02,484334,2018-01-07 15:58:26,214714.31,Shopping Basket,True,53.71
54853,13450.87,359083,2017-05-25 13:58:31,13391.86,Shopping Basket,True,59.01


In [206]:
orders_info['price_difference'].describe()

count    204691.000000
mean         -2.303391
std         297.461827
min      -90898.610000
25%          -0.010000
50%           0.000000
75%           0.000000
max       36159.900000
Name: price_difference, dtype: float64

Looks like the maximum and minimun price differences are huge: some orders are really corrupted. But we also see from the mean and the quartiles that the price difference is neglegible for most orders.

## Challenge: Remove outliers

Decide on a criteria for removing orders whenever you cannot trust the price difference between `orders` and `orderlines`. 

In [207]:
# code here

Save the data once you are sure you can trust it!

In [237]:
orderlines.to_csv('data/clean/orderlines_cl.csv', index=False)
orders.to_csv('data/clean/orders_cl.csv', index=False)

## Challenge: Cleaning products

Now it is time to clean the products dataset. Let's do a quick review of the things where wrong: 

In [209]:
print(products.info())
check_duplicates(products)
check_missing_values(products)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB
None
Duplicated rows:  8746
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64


Looking at this overview, we can see that there are different things that has to be changed: 

* Data types: 
    * price should be a float
    * promo price should be a float
* Duplicated rows. They have to be removed. 
    * To accomplish this step you can use the method `pd.DataFrame.drop_duplicates()`. Be sure you drop all the duplicates based on the column **sku**, as it is the one you will use to merge with orderlines. 
* Missing values: 
    * Description may be can be infered by the name
    * price. Is there a way we can extract the information from another table?
    * type. Do we need this column for our analysis?
    
This task can be accomoplished by using all the methods you already know.

### Start of the challenge

#### Duplicates

In [210]:
products.drop_duplicates(inplace=True)

In [211]:
products.shape

(10580, 7)

In [212]:
products.head()

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364


Check if you have a unique description for each sku: 

In [213]:
products.groupby('sku')['sku'].count().sort_values(ascending=False).head(5)

sku
APP1197    2
PAC1555    1
PAC1557    1
PAC1558    1
PAC1559    1
Name: sku, dtype: int64

The product APP1197 is not unique. Let's zoom in: 

In [214]:
products.query('sku == "APP1197"')

,sku,name,desc,price,promo_price,in_stock,type
7992,APP1197,"Apple iMac 21.5 ""Core i5 31 GHz Retina display...",Desktop Apple iMac 21.5 inch i5 31 GHz Retina ...,1729,1305.59,0,1282
8000,APP1197,"Apple iMac 21.5 ""Core i5 31 GHz Retina display...",Desktop Apple iMac 21.5 inch i5 31 GHz Retina ...,NaN,1305.59,0,1282


In [215]:
products.drop_duplicates('sku').query('sku == "APP1197"')

,sku,name,desc,price,promo_price,in_stock,type
7992,APP1197,"Apple iMac 21.5 ""Core i5 31 GHz Retina display...",Desktop Apple iMac 21.5 inch i5 31 GHz Retina ...,1729,1305.59,0,1282


I can drop the duplicates only focusing on the column 'sku': 

In [216]:
products.drop_duplicates('sku', inplace=True)

#### Data types

In [217]:
# pd.to_numeric(products['price'])

It seems this time that the problem in our products prices is different than the one we have found on orderlines. How is it possible we have more than 3 digits? 

Let's look at a random sample in our dataframe to understant more about it: 

In [218]:
products.sample(10)

,sku,name,desc,price,promo_price,in_stock,type
17162,AP20238,"Like new - Apple MacBook Pro 13 ""Core i5 25GHz...",laptop MacBook reconditioned 133 inches i5 25 ...,1199,9.455.944,0,1298
1469,KAN0036,Kanex Hub 4 USB 3.0 ports,Kanex Hub USB 3.0 Mac and PC 4 port.,59.95,429.913,0,12995397
17379,APP2145-A,"Like new - Apple iPad Pro 10.5 ""Wi-Fi 64GB Oro...",Pro New iPad 64GB Wi-Fi refitted and running l...,729,6.798.838,0,1298
16837,APP2311,"Apple MacBook Pro 13 ""Core i5 Touch Bar 31GHz ...",New MacBook Pro 13-inch Core i5 Touch Bar 31 G...,32.455.951,32.455.951,0,"1,02E+12"
16961,HOC0003-A,Open - Nike Hoco Series Apple Watch Strap 38mm...,Reconditioned sports silicone strap 38mm Apple...,45,17.969,0,2449
14589,SAT0034,Support Satechi Slim MacBook or iMac Aluminum ...,Stand with aluminum finish and practical desig...,49.99,44.99,1,8696
13433,TUC0282,Tucano ELEKTRO FLEX 8/7 Ultraslim iPhone Case ...,thin flexible sleeve with metallic shades for ...,16.9,129.906,0,11865403
17132,IFX0174,"Battery iFixit MacBook Pro 15 ""Retina Mid 2012...",Internal Battery for Macbook Pro Retina 15-inc...,142.99,134.99,1,10142
10546,PAC1604,"Apple iMac 27 ""Core i7 Retina 5K 4GHz | 8GB RA...",IMac desktop computer 27 inch Retina 5K 4GHz C...,4129,32.609.899,0,"5,74E+15"
16728,APP2259,"Apple MacBook Air 13 ""Core i7 22GHz | 8GB | 51...",laptop MacBook Air 13 inch i7 22GHz 8GB RAM an...,2029,1.934.594,0,"2,17E+11"


My strategy would be: 

1. Replace missing values with a "special" string. In that case I will put "000.000" as it would represent that this value has a wrong format and has to be changed. This step is done to avoid future problems while cleaning the column price.
2. Create a new column called "price_split" with the values of price splited by the `,`
3. Create a new column called "dots_count" counting the number of dots a price contains

In [219]:
import numpy as np

products = (
products
    # Replace all missing values by '000.000'
    .assign(price = lambda x: x['price'].replace(np.nan, '000.000'))
    .assign(price_split = lambda x: x['price'].str.split('\.'),
            dots_count = lambda x: x['price'].str.count('\.'))
)
products.head(10)

,sku,name,desc,price,promo_price,in_stock,type,price_split,dots_count
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,"[59, 99]",1
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401,[59],0
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387,[59],0
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230,[25],0
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364,"[34, 99]",1
5,APP0073,Apple Composite AV Cable iPhone and iPod white,IPhone and iPod AV Cable Dock to Composite Video.,45,420.003,0,1230,[45],0
6,KIN0008,Mac Memory Kingston 1GB 667MHz DDR2 SO-DIMM,1GB RAM Mac mini and iMac (2006/07) MacBook Pr...,18.99,146.471,0,1364,"[18, 99]",1
7,KIN0009,Mac Memory Kingston 2GB 800MHz DDR2 SO-DIMM,2GB RAM iMac with Intel Core 2 Duo (Penryn).,36.99,274.694,0,1364,"[36, 99]",1
8,KIN0001-2,Mac memory Kingston 4GB (2x2GB) 667MHz DDR2 SO...,RAM 4GB (2x2GB) Mac mini and iMac (2006/07) Ma...,74,669.904,0,1364,[74],0
9,APP0100,Apple Adapter Mini Display Port to VGA,Adapter Mini Display Port to VGA MacBook and M...,35,330.003,0,1325,[35],0


In [220]:
products.query('dots_count == 2')

,sku,name,desc,price,promo_price,in_stock,type,price_split,dots_count
665,CRU0015-2,Crucial memory Mac 16GB (2x8GB) SO-DIMM DDR3 1...,RAM 16GB (2x8GB) 135V MacBook Pro iMac (2012/2...,1.639.792,1.629.894,1,1364,"[1, 639, 792]",2
792,APP0672,Apple iPhone 5S 16GB Space Gray,New iPhone 5S 16G Libre (ME432Y / AB).,4.694.994,4.694.994,0,NaN,"[4, 694, 994]",2
797,APP0673,Apple iPhone 5S 16GB Silver,New Free iPhone 5S 16GB (ME433Y / A).,4.090.042,4.090.042,0,NaN,"[4, 090, 042]",2
827,PAC0339,NewerTech miniStack 4TB Hard Drive Mac,External Box Hard Drive Mac + 4TB.,2.199.791,2.199.901,0,11935397,"[2, 199, 791]",2
885,PAC0376,OWC Mercury Elite Pro Dual Thunderbolt + 8TB,RAID outer box 35 inch SATA connection Thunder...,5.609.698,5.549.895,0,11935397,"[5, 609, 698]",2
...,...,...,...,...,...,...,...,...,...
19312,REP0424,Input repair Headphones iPad,Repair service including parts and labor for iPad,6.999.003,69.99,0,"1,44E+11","[6, 999, 003]",2
19313,REP0421,iPad charging connector repair,Repair service including parts and labor for iPad,6.999.003,69.99,0,"1,44E+11","[6, 999, 003]",2
19314,REP0416,iPad front camera repair,Repair service including parts and labor for iPad,6.999.003,69.99,0,"1,44E+11","[6, 999, 003]",2
19315,REP0413,repair rear camera iPad,Repair service including parts and labor for iPad,6.999.003,69.99,0,"1,44E+11","[6, 999, 003]",2


Now I need to look at each value in the new column "price_split" and check the folliwing condition: 

If the last element of the value in price split has a length bigger than 2 then add a `True` inside the list `need_check`, else add a `False`. 

Example 1: 

 - Input: price_split = `[[59],[99]]` The last element of the value in price split is `[99]`. The length of this element is 2, as it only have 2 characters. Then we will append a `False` on the list `need_check`. 

Exemple 2: 

- Input: price_split = `[[234],[895]]` The last element of the value in price split is `[895]`. The length of the last element is 3, as it has 3 characters. Then we will append a `True` on the list `need_check`.

In [221]:
need_check = []
len_split = []
for val in products['price_split']:    
    len_split.append(len(val))
    if len(val[-1]) > 2: 
        need_check.append(True)
    else:
        need_check.append(False)

We have created two lists: 

* `need_check`: tells us which element of the dataframe has a last element on `price_split` with a lenght higher than 2. 
* `len_split` : tells us how many elements are inside each `price_split`. 
    * Example 1: `[[59],[99]]` it has a lenght of 2
    * Example 2: `[59]` it has a lenght of 1
    * Example 3: `[[654],[998]]` it has a lenght of 2
    * Example 4: `[[9],[654],[998]]` it has a lenght of 3
    
Then we can add this elements into our products dataframe to detect these rows: 

In [222]:
(
products
    .assign(need_check = need_check, 
            len_split = len_split)
    .query('need_check==True & len_split > 1 & dots_count != 0')
).head()

,sku,name,desc,price,promo_price,in_stock,type,price_split,dots_count,need_check,len_split
34,TWS0019,Twelve South MagicWand support Apple Magic Tra...,MagicWand for wireless keyboard and Magic Trac...,000.000,299.899,0,8696,"[000, 000]",1,True,2
362,REP0043,Speaker lower repair iPhone 4,Repair service including parts and labor for i...,499.004,499.004,0,"1,44E+11","[499, 004]",1,True,2
480,PIE0011,Internal Battery for iPhone 3G,Replacement AC Adapter for Apple iPhone 3G.,98.978,98.978,0,21485407,"[98, 978]",1,True,2
515,SEN0061,Sennheiser EZX 80 Handsfree iPhone iPad and iP...,IPhone bluetooth headset with microphone iPad ...,649.891,649.891,0,5384,"[649, 891]",1,True,2
518,SEV0026,Service installation RAM + HDD + SSD MacBook /...,RAM + HDD installation + SSD in your MacBook /...,599.918,599.918,0,20642062,"[599, 918]",1,True,2


By defining a set of rules, I can quickly detect the prices that are giving me errors. Those rules are: 

* The length of the last element on the value `price_split` has a lenght bigger than 2 (This is what the column `need_check` is telling us).
* There is more than 1 element inside the column `price_split` (Information given by the column `len_split`). 
* The number of dots in price is different than 0 (Column `dots_count`). 

The next steps would be to use this conditions to replace this values by missing values. To do it, I am going to use the function from the module `numpy` called `np.where`. It works as an `if else` statement; if a condition is accomplished, then return a value, else, return another. 

In [223]:
products = (
products
    .assign(need_check = need_check, 
            len_split = len_split)
    .assign(
        price = lambda x: pd.to_numeric(
            np.where((x['need_check']==True) & (x['len_split'] > 1) & (x['dots_count'] != 0), np.nan, x['price'])
        )
    )
)
products.query('need_check==True & len_split > 1 & dots_count != 0').sample(10)

,sku,name,desc,price,promo_price,in_stock,type,price_split,dots_count,need_check,len_split
12859,PAC1479,Synology DS216 + II | Seagate 8TB IronWolf for...,Synology DS216 + II + 8TB (2x4TB) IronWolf Sea...,NaN,5.921.788,0,12175397,"[6, 929, 696]",2,True,3
14923,PAC1902,SSD Crucial MX300 expansion kit 1TB + 8GB 1600...,SSD expansion 1TB + 8GB 1600Mhz RAM for Mac mi...,NaN,384.584,1,1433,"[4, 463, 085]",2,True,3
18726,TAD0002-A,Open - Tado V2 Intelligent Heating Thermostat,Smart Thermostat refurbished and bridge with a...,NaN,1.512.253,0,11905404,"[1, 799, 899]",2,True,3
15396,REP0363,Cases mini repair posterior chamber,Repair service including parts and labor for i...,NaN,69.99,0,"1,44E+11","[6, 999, 003]",2,True,3
15992,PAC1983,Synology DS1817 + | 16GB RAM | 64TB (8x8TB) Se...,NAS with 16GB of RAM and 64TB for Mac and PC,NaN,2.905.746,0,12175397,"[40, 377, 892]",2,True,3
15420,REP0311,Wi-Fi antenna repair iPhone 6 Plus,Repair service including parts and labor for i...,NaN,599.906,0,"1,44E+11","[599, 906]",1,True,2
11270,PAC1669,QNAP TS-453A | 16GB RAM | Seagate 12TB Iron Wolf,NAS QNAP TS-453A with 16GB of RAM memory + 12T...,NaN,12.163.682,1,12175397,"[12, 332, 382]",2,True,3
15991,PAC1982,Synology DS1817 + | 16GB RAM | 48TB (8x6TB) Se...,NAS with 16GB of RAM and 48TB for Mac and PC,NaN,26.417.458,0,12175397,"[33, 977, 892]",2,True,3
14942,PUR0136-A,Open - Puro UltraSlim Case 03 + Protector iPho...,03mm thin cover with screen protector included...,NaN,88.977,0,1298,"[129, 906]",1,True,2
16835,APP2308,"Apple MacBook Pro 13 ""Core i5 Touch Bar 33GHz ...",New MacBook Pro 13 inch Touch Bar 33 GHz Core ...,NaN,31.255.946,0,"1,02E+12","[31, 255, 946]",2,True,3


As you can see, we replaced all the values with wrong prices with missing data. New let's drop all the columns we do not need: 

In [224]:
products.drop(['price_split','need_check','len_split','dots_count'], axis=1, inplace=True)
check_missing_values(products)

Missing values:
sku              0
name             0
desc             7
price          587
promo_price      0
in_stock         0
type            50
dtype: int64


And now is time to find a strategy to fill this missing values. 

In [225]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10579 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          10579 non-null  object 
 1   name         10579 non-null  object 
 2   desc         10572 non-null  object 
 3   price        9992 non-null   float64
 4   promo_price  10579 non-null  object 
 5   in_stock     10579 non-null  int64  
 6   type         10529 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 661.2+ KB


#### Missing values

As now we have incremented the number of missing data (we transformed all the wrong prices to missing ones) let's find out an strategy to fill them. 

But first, let's look if they are important in our dataset:

In [226]:
prod_na = products.loc[products.isna().sum(axis=1) > 0,:].copy()
prod_na

,sku,name,desc,price,promo_price,in_stock,type
34,TWS0019,Twelve South MagicWand support Apple Magic Tra...,MagicWand for wireless keyboard and Magic Trac...,NaN,299.899,0,8696
307,SAN0017,SanDisk Cruzer Edge USB 2.0 Flash Drive 16GB,Pendrive USB 2.0 Mac and PC.,8.0,59.895,0,NaN
362,REP0043,Speaker lower repair iPhone 4,Repair service including parts and labor for i...,NaN,499.004,0,"1,44E+11"
480,PIE0011,Internal Battery for iPhone 3G,Replacement AC Adapter for Apple iPhone 3G.,NaN,98.978,0,21485407
515,SEN0061,Sennheiser EZX 80 Handsfree iPhone iPad and iP...,IPhone bluetooth headset with microphone iPad ...,NaN,649.891,0,5384
...,...,...,...,...,...,...,...
19312,REP0424,Input repair Headphones iPad,Repair service including parts and labor for iPad,NaN,69.99,0,"1,44E+11"
19313,REP0421,iPad charging connector repair,Repair service including parts and labor for iPad,NaN,69.99,0,"1,44E+11"
19314,REP0416,iPad front camera repair,Repair service including parts and labor for iPad,NaN,69.99,0,"1,44E+11"
19315,REP0413,repair rear camera iPad,Repair service including parts and labor for iPad,NaN,69.99,0,"1,44E+11"


Decide which percentage of the data represents. Always think smart about using your time: 

* Which % of the orders contains one of these products? In that case I has to look at orderlines.

In [227]:
orderlines.shape[0]

293743

In [228]:
# percentage of orders with products with missing prices on products dataset
orderlines.loc[orderlines['sku'].isin(prod_na['sku']),:].shape[0] / orderlines.shape[0]

0.025604014393534484

In [229]:
# this is interesting, as for the products data the percentage is much smaller
prod_na.shape[0] / products.shape[0]

0.06040268456375839

Which impact has on the total revenue?

In [230]:
orderlines.loc[orderlines['sku'].isin(prod_na['sku']),:]['total_price'].sum() / orderlines['total_price'].sum()

0.027227081578374515

With a 3% only on the total revenue, but we are considering all the orders, and we are interested on the completed ones. 

In [231]:
na_skus = prod_na['sku'].tolist()

(
orderlines
    .merge(orders, how='left', left_on='id_order', right_on='order_id')
    .query('state == "Completed" & sku == @na_skus')
    ['total_price'].sum()
) / (
orderlines
    .merge(orders, how='left', left_on='id_order', right_on='order_id')
    .query('state == "Completed"')
    ['total_price'].sum()
)

0.02922734355997912

As you can see, the products that are missing only represents 2.9% of the total invoice. 
I will not drop them, as it still has some information which I would like to keep it to create categories (sku, item name and item description), so I am going to save it with prices as missing values. But I will add an extra column telling me which is the maximum price for these skus in orderlines.

In [232]:
# find out which sku are missing in products
prod_na_list = (
products
    .loc[products['price'].isna(),'sku']
    .tolist()
)

# find them on orderlines, and aggregate the information to find the maximum value
orderlines_prod_na_max = (
orderlines
    .query("sku == @prod_na_list")
    .groupby('sku') 
    .agg({'unit_price':'max'})
    .rename(columns={'unit_price':'max_price_orderlines'})
    .reset_index()
)
orderlines_prod_na_max[:5]

,sku,max_price_orderlines
0,ADN0023-A,16.81
1,APP0660-A,253.11
2,APP0869,539.00
3,APP0870,539.00
4,APP0871,539.00


Time to add the column `max_price_orderlines` in our products data:

In [233]:
products = (
products
    .merge(orderlines_prod_na_max, how='left',on='sku')
)
products.loc[products['max_price_orderlines'].notna()].sample(10)

,sku,name,desc,price,promo_price,in_stock,type,max_price_orderlines
1269,PAC0570,Kingston V300 SSD expansion kit 120GB + OWC Mo...,120GB SSD expansion kit + Adapter Mount Pro fo...,NaN,925.844,0,1433,87.99
10091,CAL0010,Station 3 Caldigit Thunderbolt Thunderbolt out...,Thunderbolt external box 3 to magnify your tea...,NaN,2.999.905,1,NaN,299.99
9649,FIB0008,Fibaro HomeKit Sensor Door / Window,multi-function sensor for doors and windows co...,NaN,599.918,1,11859,59.99
3410,KAN0022-A,Open - Kanex adapter Thunderbolt to Ethernet +...,Gigabyte Thunderbolt adapter cable to Ethernet...,NaN,903.296,0,1298,90.33
4985,PAC1483,Synology DS216 + II | WD 8TB Network for Mac a...,Synology DS216 + II + 8TB (2x4TB) WD Red hard ...,NaN,6.121.789,0,12175397,650.99
6663,PUR0144-A,"Open - Pure Nude Ultraslim 03 ""7 Clear iPhone ...",flexible transparent cover with 03mm thick iPh...,NaN,91.828,0,1298,9.18
6878,REP0264,volume button repair on vibration control and ...,Repair service including parts and labor for i...,NaN,599.906,0,"1,44E+11",59.99
7676,WIT0011-A,Open - Withings Blood Pressure Monitor Wireles...,Wireless Withings Blood Pressure Monitor Blood...,NaN,921.976,0,1298,92.20
6966,REP0268,Wi-Fi antenna repair iPhone 4S,Repair service including parts and labor for i...,NaN,399.905,0,"1,44E+11",39.99
368,SEV0027,Budget Request repair Apple MacBook Air,diagnosis to repair MacBook Air.,NaN,199.892,0,"1,46E+11",19.99


And not let's store it as a products clean csv into our path: 

In [234]:
products.drop(['promo_price','in_stock','type'], axis=1).to_csv(path + 'products_cl.csv', index=False)

## Brands

The brands csv looks fine, so we can work together with it. 

In [167]:
brands.to_csv(path + 'brands_cl.csv', index=False)